# Potential Data Creation

Given a periodic potential v, symbolically calculate significant data, like characteristic polynomials etc

We work with the monodromy matrix, as it is easier to calculate. Cf. Remark 3.9

In particular, $M_{1,1} = \pm \chi_{A_{0..K-1}}(E)$ and $M_{1,2} = \pm \chi_{A_{0..K-2}}(E)$.

In [ ]:
l = var('l')
E = var('E')
    
def cutoff(v,K):
#
# determine the cutoff matrix H_{0..K} with potential v[0:K-1]
#
    A = matrix(SR,K+1,K+1) #start enumerating from 0
    A = A + matrix.toeplitz([0,1] + [0]*(K-1), [1]+[0]*(K-1))
    for k in range(K+1):
        A[k,k] = v[k]
    return A

def mon(v):
#
# Calculate the general monodromy matrix with scaling l and energy E
#
    M = identity_matrix(2)
    for vv in v:
        M = matrix(2,2,[E - l*vv, -1, 1, 0])*M
    return M

def lim_pots(v):
#
# determine all limit potentials of a given periodic potential v
#
    v_rev = deepcopy(v)
    lim = shift_pots(v)
    v_rev.reverse()
    lim_rev = shift_pots(v_rev)
    for pot in lim_rev:
        if pot not in lim:
            lim.append(pot)
    return lim
    
def shift_pots(v):
#
# determine all shifts of a given periodic potential v
#
    shift = []
    for c in range(len(v)):
        if v not in shift:
            shift.append(v)
        v = right_shift(v)
    return shift
    
def right_shift(a):
    return [a[-1] , *a[:-1]]

# Consider E=0

Given period length, systematically check for every monodromy matrix if there exists a scaling factor $l$ such that $M_{2,1} = 0$. If such scalings exist, check whether the full operator is also not invertible by verifying the trace condition.

In [ ]:
def analyse_v(v):
    data = {}
    data['mon'] = mon(v).expand()
    sols = solve(data['mon'][1][0].subs(E=0) == 0, l)
    data['sol'] = list(map(lambda s: s.rhs(), sols))
    data['sol'].sort()
    data['trace'] = list(map(lambda s: data['mon'].subs(E=0).trace().subs(l=s), data['sol']))
    return data

sigma = [0,1]
pots = {}
for K in range(3,10):
    v_list = Tuples(sigma,K).list()
    for v in v_list:
        pots[tuple(v)] = deepcopy(analyse_v(v))

In [ ]:
v_list = Tuples(sigma,K).list()

for v in v_list:
    if true in [abs(t).n() > 2 for t in pots[tuple(v)]['trace']]: #uncomment this line for all candidates
        if true in [s in QQ for s in pots[tuple(v)]['sol']]: #uncomment this line for all roots
            print([v, [s for s in pots[tuple(v)]['sol']],  [s.n() for s in pots[tuple(v)]['trace']]])